In [95]:
import pandas as pd
import os

os.chdir("../../data/raw_data/gems/")

FileNotFoundError: [Errno 2] No such file or directory: '../../data/raw_data/gems/'

In [96]:
ls

2018_I10gem.csv                 fun.py
2018_I10gem.txt                 icd10OrderFiles.pdf
2018_I9gem.csv                  icd10_descriptions.zip
2018_I9gem.txt                  icd10_gems.zip
CMS32_DESC_LONG_DX.csv          icd10_gems_lookup.csv
CMS32_DESC_LONG_DX.txt          icd10cmCodesFile.pdf
CMS32_DESC_LONG_SG.txt          icd10cm_codes_2018.csv
CMS32_DESC_LONG_SHORT_DX.xlsx   icd10cm_codes_2018.txt
CMS32_DESC_LONG_SHORT_SG.xlsx   icd10cm_codes_addenda_2018.txt
CMS32_DESC_SHORT_DX.txt         icd10cm_order_2018.txt
CMS32_DESC_SHORT_SG.txt         icd10cm_order_addenda_2018.txt
Dxgem_guide_2018.pdf            icd9_descriptions.zip
GemsTechDoc_2018.pdf            icd9_gems_lookup.csv
__pycache__/                    sql_test.csv


In [204]:
filename = "2018_I10gem.txt"

In [205]:
filename = f"{filename.split('.')[0]}.csv"

In [206]:
filename

'2018_I10gem.csv'

In [84]:
def format_icd(filename: str):
    
    df = pd.read_csv("2018_I10gem.txt", 
                     sep="[\s]{1,}",
                     names=["source", "target", "flags"],
                     encoding="latin-1")
    
    save_name = update_file_name(filename)
    df.to_csv(save_name)
    
    
def update_file_name(filename: str):
    
    return f"{filename.split('.')[0]}.csv"
    
    

def format_desc(filename: str, codetype: str, separater: str = ","):
    
    df = pd.read_csv(filename,names=[codetype], sep = separater, encoding="latin-1")
    
    df[[codetype, f"{codetype}_desc"]] = df[codetype].str.split(" ", 1, expand=True)
    
    save_name = update_file_name(filename)
    df.to_csv(save_name)


In [85]:
format_icd("2018_I10gem.txt")
format_icd("2018_I9gem.txt")

format_desc("CMS32_DESC_LONG_DX.txt", "icd9", separater = ",")
format_desc("icd10cm_codes_2018.txt", "icd10", separater = "//s")

/Users/patricklong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/Users/patricklong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [213]:
ten_gems = pd.read_csv("2018_I10gem.csv")

In [215]:
ten_gems.head()

,Unnamed: 0,source,target,flags
0,0,A000,0010,0
1,1,A001,0011,0
2,2,A009,0019,0
3,3,A0100,0020,10000
4,4,A0101,0020,10000


In [216]:
nine = pd.read_csv("icd9_gems_lookup.csv")

In [217]:
ten = pd.read_csv("icd10_gems_lookup.csv")

In [218]:
nine.head()

,source,target,flags,icd10,description
0,0010,A000,0,A000,"Cholera due to Vibrio cholerae 01, biovar cholerae"
1,0011,A001,0,A001,"Cholera due to Vibrio cholerae 01, biovar eltor"
2,0019,A009,0,A009,"Cholera, unspecified"
3,0020,A0100,10000,A0100,"Typhoid fever, unspecified"
4,0021,A011,0,A011,Paratyphoid fever A


In [244]:
ten.head(10)

,source,target,flags,icd9,description
0,A000,0010,0,0010,Cholera due to vibrio cholerae
1,A001,0011,0,0011,Cholera due to vibrio cholerae el tor
2,A009,0019,0,0019,Cholera
3,A0100,0020,10000,0020,Typhoid fever
4,A0101,0020,10000,0020,Typhoid fever
5,A0102,0020,10000,0020,Typhoid fever
6,A0103,0020,10000,0020,Typhoid fever
7,A0104,0020,10000,0020,Typhoid fever
8,A0105,0020,10000,0020,Typhoid fever
9,A0109,0020,10000,0020,Typhoid fever


In [237]:
pd.set_option('display.max_colwidth', 255)


def icd9_to_icd10(icd_code: str):

    lookup_table = pd.read_csv("icd9_gems_lookup.csv")

    return lookup_table[lookup_table["source"] == icd_code].loc[:,["source", "icd10", "description"]]


def icd10_to_icd9(icd_code: str):
    
    lookup_table = pd.read_csv("icd10_gems_lookup.csv")
    
    return lookup_table[lookup_table["source"] == icd_code].loc[:,["source", "icd9", "description"]]
    


In [247]:
lookup_table = pd.read_csv("icd9_gems_lookup.csv")

In [263]:
lookup_table["description"] = lookup_table["description"].str.upper()

In [267]:
lookup_table[lookup_table["description"].str.contains("VIB")]

,source,target,flags,icd10,description
0,0010,A000,0,A000,"CHOLERA DUE TO VIBRIO CHOLERAE 01, BIOVAR CHOLERAE"
1,0011,A001,0,A001,"CHOLERA DUE TO VIBRIO CHOLERAE 01, BIOVAR ELTOR"
29,0054,A053,0,A053,FOODBORNE VIBRIO PARAHAEMOLYTICUS INTOXICATION
30,00581,A055,0,A055,FOODBORNE VIBRIO VULNIFICUS INTOXICATION
10481,7084,L504,0,L504,VIBRATORY URTICARIA


In [286]:
def search_icd10(search_term: str):

    lookup_table = pd.read_csv("icd9_gems_lookup.csv")
    
    lookup_table["description"] = lookup_table["description"].str.upper()    

    return lookup_table[lookup_table["description"].str.contains(search_term.upper())].loc[:,["icd10", "description"]]



def search_icd9(search_term: str):

    lookup_table = pd.read_csv("icd10_gems_lookup.csv")
    
    lookup_table["description"] = lookup_table["description"].str.upper()    

    return lookup_table[lookup_table["description"].str.contains(search_term.upper())].loc[:,["icd9", "description"]]


In [288]:
search_icd10("vib")

,icd10,description
0,A000,"CHOLERA DUE TO VIBRIO CHOLERAE 01, BIOVAR CHOLERAE"
1,A001,"CHOLERA DUE TO VIBRIO CHOLERAE 01, BIOVAR ELTOR"
29,A053,FOODBORNE VIBRIO PARAHAEMOLYTICUS INTOXICATION
30,A055,FOODBORNE VIBRIO VULNIFICUS INTOXICATION
10481,L504,VIBRATORY URTICARIA


In [246]:
print(icd10_to_icd9("R6521"))

      source   icd9    description
35983  R6521  99592  Severe sepsis
35984  R6521  78552   Septic shock


In [212]:
def join_icd_desc(df_icd,df_desc, key: str):
    df = pd.merge(df_icd, df_desc,
                  left_on = "target",
                 right_on = key)
    return df


def save_icd_desc_tables(df, df_name: str):
    df.to_csv(df_name, index = False)

    
def add_icd_desc(df_icd9, df_icd9_desc, df_icd10, df_icd10_desc):
    
    df9_lookup = join_icd_desc(df_icd9,
                          df_icd10_desc,
                          key = "icd10")
    
    
    df10_lookup = join_icd_desc(df_icd10,
                           df_icd9_desc,
                           key = "icd9")
